<a href="https://colab.research.google.com/github/harvard-visionlab/sroh/blob/main/2022/sroh_2022_feature_extraction_test_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract Features For a Batch of Test Images

## downloads

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
!wget -c https://www.dropbox.com/s/wubxg3tslwruqpb/face_stimuli_png.zip
!wget -c https://raw.githubusercontent.com/harvard-visionlab/sroh/main/2022/imagenet_categories.py
!wget -c https://raw.githubusercontent.com/harvard-visionlab/sroh/main/2022/feature_extractor.py

In [ ]:
!unzip face_stimuli_png.zip

In [ ]:
from glob import glob 

files = sorted(glob('./face_stimuli_png/*.png'))
files

In [ ]:
from PIL import Image 

Image.open('./face_stimuli_png/blurred_white_male.png')

# do feature extraction and comparison

In [ ]:
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
  transforms.Resize(224),
  transforms.CenterCrop(224), 
  transforms.ToTensor(),
  normalize,
])
transform

In [ ]:
from torchvision import models 

model = models.alexnet(pretrained=True)
model

## compute similarities by model layer

In [ ]:
import torch
import numpy as np 
import pandas as pd 
import seaborn as sns 
from pathlib import Path 
from PIL import Image 
from fastprogress import progress_bar
from feature_extractor import FeatureExtractor, get_layer_names

@torch.no_grad()
def compare_features(model, layer_names, batch):
  model.eval()
  RSM = {}
  with FeatureExtractor(model, layer_names) as extractor:
    features = extractor(batch)
  
  for layer_name,X in progress_bar(features.items()):
    RSM[layer_name] = torch.corrcoef(X.flatten(1))
  
  return RSM

In [ ]:
layer_names = get_layer_names(model)
layer_names

In [ ]:
images = [
 './face_stimuli_png/control_white_male.png',
 './face_stimuli_png/inverted_white_male.png',
 './face_stimuli_png/sheared_white_male.png',
]
imgs = [Image.open(img).convert('RGB') for img in images]
batch = torch.stack([transform(img) for img in imgs])
print(batch.shape)

In [ ]:
filenames = [Path(fname).name for fname in images]
filenames

In [ ]:
pairs = [
  (0,1,'control-vs-inverted'),
  (0,2,'control-vs-sheared'),
]
pairs

In [ ]:
imgs[0]

In [ ]:
imgs[1]

In [ ]:
imgs[2]

In [ ]:
RSM = compare_features(model, layer_names, batch)

In [ ]:
columns = ['layer_num', 'layer_name', 'image1', 'image2', 'pair', 'pearsonr']
df = pd.DataFrame(columns=columns)
df

In [ ]:
from collections import defaultdict
from pathlib import Path 

results = defaultdict(list)

for layer_num,(layer_name,matrix) in enumerate(RSM.items()):
  for idx1,idx2,pair in pairs:
    image1 = filenames[idx1]
    image2 = filenames[idx2]
    pearsonr = matrix[idx1,idx2].item()

    results['layer_num'].append(layer_num)
    results['layer_name'].append(layer_name)
    results['image1'].append(image1)
    results['image2'].append(image2)
    results['pair'].append(pair)
    results['pearsonr'].append(pearsonr)

df = pd.DataFrame(results, columns=columns)
df

In [ ]:
# df[df.layer_num==0]

In [ ]:
ax = sns.lineplot(data=df, x="layer_num", y="pearsonr", hue="pair")
ax.set_ylim([0,1.0])